In [ ]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception

# Tải mô hình Xception đã huấn luyện sẵn nhưng không có các lớp phân loại
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Đóng băng các lớp của mô hình gốc để không huấn luyện lại
base_model.trainable = False

# Xây dựng mô hình cuối cùng
model = Sequential([
    base_model,  # Thêm mô hình Xception vào
    GlobalAveragePooling2D(),  # Áp dụng Global Average Pooling
    Dense(128, activation='relu'),  # Lớp Dense cho phân loại
    Dense(1, activation='sigmoid')  # Lớp phân loại nhị phân
])

# Biên dịch mô hình
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Định nghĩa ImageDataGenerators để tăng cường dữ liệu
train_datagen = ImageDataGenerator(
    rescale=1. / 255,  # Chuẩn hóa ảnh về [0, 1]
    rotation_range=40,  # Xoay ngẫu nhiên
    width_shift_range=0.2,  # Dịch ngang ngẫu nhiên
    height_shift_range=0.2,  # Dịch dọc ngẫu nhiên
    shear_range=0.2,  # Biến dạng ngẫu nhiên
    zoom_range=0.2,  # Phóng to/thu nhỏ ngẫu nhiên
    horizontal_flip=True,  # Lật ngang ngẫu nhiên
    fill_mode='nearest'  # Điền pixel khi biến đổi
)

test_datagen = ImageDataGenerator(rescale=1. / 255)  # Không tăng cường dữ liệu cho tập kiểm thử

# Đường dẫn đến thư mục chứa dữ liệu huấn luyện và kiểm thử
train_dir = 'F:/DACNTT/Report/extracted_train_frames'  # Thư mục ảnh huấn luyện
validation_dir = 'F:/DACNTT/Report/extracted_test_frames'  # Thư mục ảnh kiểm thử

# Kiểm tra thư mục con (fake, real) tồn tại
def check_subfolders(directory):
    for subfolder in ['fake', 'real']:
        path = os.path.join(directory, subfolder)
        if not os.path.exists(path):
            raise FileNotFoundError(f"Thư mục '{path}' không tồn tại! Vui lòng kiểm tra dữ liệu.")

check_subfolders(train_dir)
check_subfolders(validation_dir)

# Tạo các generator cho việc tải ảnh
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),  # Kích thước ảnh phù hợp với Xception
    batch_size=32,
    class_mode='binary'  # Phân loại nhị phân
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)

# Huấn luyện mô hình
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # Số bước mỗi epoch
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20
)

# Lưu mô hình đã huấn luyện
model.save('deepfake_detection_model.h5')
print("Mô hình đã được lưu thành công!")

# Vẽ đồ thị huấn luyện
import matplotlib.pyplot as plt

# Đồ thị Accuracy
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Đồ thị Loss
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


Found 3180 images belonging to 2 classes.
Found 3300 images belonging to 2 classes.


f:\Anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
 3/99 ━━━━━━━━━━━━━━━━━━━━ 5:25 3s/step - accuracy: 0.6788 - loss: 0.6326

KeyboardInterrupt: 